In [ ]:
from IPython.display import display
from datasets import load_dataset
from peft import prepare_model_for_kbit_training
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model
from transformers import (
from transformers import AutoTokenizer
import ipywidgets as widgets
import os
import torch
import wandb

Overall Plan
Stage	Description
1.	Install and import dependencies
2.	Load and prepare your dataset
3.	Tokenize the dataset
4.	Load model with 4-bit quantization
5.	Prepare model for PEFT fine-tuning
6.	Set up training configuration
7.	Train the model

In [ ]:
!pip install peft accelerate bitsandbytes transformers datasets


In [ ]:
!pip uninstall -y bitsandbytes
!pip install bitsandbytes -U --prefer-binary --no-cache-dir


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 317.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 275.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 273.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 269.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 333.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 263.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 284.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 306.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 267.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 329.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 340.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82

In [ ]:
!nvidia-smi

Mon Apr  7 19:05:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   44C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
print(torch.cuda.is_available())  # Should return True


True


**STAGE 1: Install & Import Dependencies**

In [ ]:
!pip install transformers peft datasets accelerate bitsandbytes wandb --quiet

    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.


**STAGE 2: Load and Prepare Dataset**

We’ll load the nist-research/bodmas Cybersecurity Dataset from Hugging Face and use the report field for training:with text structured like this:

In [ ]:

# Load the ZeroShot Cybersecurity Corpus
dataset = load_dataset("zeroshot/cybersecurity-corpus")

# Preview the dataset
print(dataset)
print(dataset["train"][0])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sent_train.csv:   0%|          | 0.00/112k [00:00<?, ?B/s]

sent_valid.csv:   0%|          | 0.00/29.7k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/789 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/211 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 789
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 211
    })
})
{'text': 'U.S. Air Force Announces Third Bug Bounty Program - https://t.co/DVDtbF6iKI', 'label': 0}


**STAGE 3: Tokenize Dataset**

In [ ]:

# Load the TinyLlama tokenizer
checkpoint = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# Apply the tokenizer to the dataset
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Optional: Remove the 'text' column if no longer needed
tokenized_dataset = tokenized_dataset.remove_columns(["text"])

# Set the dataset format for PyTorch
tokenized_dataset.set_format("torch")


Map:   0%|          | 0/211 [00:00<?, ? examples/s]

**STAGE 4: Load 4-bit Quantized Model**

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

base_model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

**STAGE 5: Prepare Model for PEFT**

In [ ]:
model = prepare_model_for_kbit_training(base_model)


**STAGE 6: Set Up Training Arguments**

In [ ]:
wandb.login()  # or use wandb.init(project="your_project")

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    fp16=True,
    report_to="wandb",
    run_name="tinyllama-cybersecurity-finetune",
    save_total_limit=2,
    push_to_hub=False,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: okonp07 (okonp07-psp-analytics-limited) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


**7. Prepare your LoRA configuration (Pre-Training)**

In [ ]:

# Prepare the quantized model for LoRA fine-tuning
model = prepare_model_for_kbit_training(base_model)

# Define your LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # depends on model architecture
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

# (Optional) Print trainable params
model.print_trainable_parameters()


trainable params: 1,126,400 || all params: 1,101,174,784 || trainable%: 0.1023


**STAGE 7: Train the Model**

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()


<ipython-input-15-1678fec8a42c>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss
0,3.311800,3.446419
1,3.217900,3.334044
2,3.160900,3.304780


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=294, training_loss=3.289771401152319, metrics={'train_runtime': 470.4559, 'train_samples_per_second': 5.031, 'train_steps_per_second': 0.625, 'total_flos': 7495572403519488.0, 'train_loss': 3.289771401152319, 'epoch': 2.992405063291139})

**STEP 9: (Optional) Save Model**

Remove the hashes and run the cell below to save the model

In [ ]:
trainer.save_model("./final_model")
tokenizer.save_pretrained("./final_model")


**Done!**


We now have a complete end-to-end fine-tuning pipeline for a small 1.1B parameter model using 4-bit quantization. It’s Colab-A100 safe and W&B-ready.

Let me know if you'd like to add:

LoRA for efficient fine-tuning

Push to HuggingFace

Text generation code for testing

Custom dataset formatting

**2. Query the model (inference)**
Here’s how you can now test it with prompts:

In [ ]:
def generate_response(question):
    prompt = f"Q: {question}\nA:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        # Generating output with better sampling options
        outputs = model.generate(
            **inputs,
            max_length=200,      # Allow longer responses
            do_sample=True,      # Enable sampling for creativity
            top_p=0.95,          # Top-p (nucleus sampling)
            top_k=50,            # Top-k for diversity
            temperature=0.8,     # Control randomness in response
            pad_token_id=tokenizer.eos_token_id  # Avoid padding token errors
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Look for the part after "A:" for the actual response
    if "A:" in decoded:
        response = decoded.split("A:")[1].strip()
    else:
        response = decoded.strip()

    return response


In [ ]:

# Assuming these are already defined earlier in the notebook
# model = your fine-tuned model object
# tokenizer = the corresponding tokenizer

# Widgets for input and output
question_box = widgets.Textarea(
    value='',
    placeholder='Type your question here...',
    description='Question:',
    layout=widgets.Layout(width='100%', height='100px'),
    style={'description_width': 'initial'}
)

output_box = widgets.Output()

# What happens on submit
def on_click_generate(b):
    output_box.clear_output()
    question = question_box.value.strip()
    if question:
        with output_box:
            print("Question:", question)
            print("Answer:", generate_response(question))

# Button to submit the question
ask_button = widgets.Button(
    description="Ask",
    button_style='primary',
    tooltip="Click to get the model's answer"
)
ask_button.on_click(on_click_generate)

# Display everything
display(question_box, ask_button, output_box)


Textarea(value='', description='Question:', layout=Layout(height='100px', width='100%'), placeholder='Type you…

Button(button_style='primary', description='Ask', style=ButtonStyle(), tooltip="Click to get the model's answe…

Output()